<a href="https://colab.research.google.com/github/ZwL6/MZT/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=aef2ad17b293bc9448c317468197c5a0523f85344e3e1137f55dcce3de030f79
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 13 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:a

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
# Get cities file
!wget https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json



--2023-06-28 14:33:02--  https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15829 (15K) [text/plain]
Saving to: ‘cities.json’

cities.json         100%[===================>]  15.46K  --.-KB/s    in 0.001s  

2023-06-28 14:33:02 (27.3 MB/s) - ‘cities.json’ saved [15829/15829]



In [4]:
#making standerd route
import json
import random

# Set seed
random.seed(1)
with open('cities.json', 'r') as file:
    data = json.load(file)

cities = [d['city'] for d in data]

merchandise = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

routes = []

for i in range(1000):
    route = {}
    route['id'] = i

    # Generate a list of non-repeating random cities
    city_sample = random.sample(cities, 3)

    # Randomly generate the number of merchandise
    num_merchandise = random.randint(1, 10)

    # Randomly select merchandise types
    merchandise_sample = random.sample(merchandise, random.randint(3, 5))

    # Build the route dictionary format
    route['route'] = []
    for j in range(2):
        route_info = {}
        route_info['from'] = city_sample[j]
        route_info['to'] = city_sample[j+1]

        # Generate random merchandise quantities for each route
        merchandise_dict = {}
        for m in merchandise_sample:
            merchandise_dict[m] = random.randint(1, num_merchandise)

        route_info['merchandise'] = merchandise_dict

        route['route'].append(route_info)

    routes.append(route)

# Write the route list to a JSON file without corrupt records
valid_routes = [r for r in routes if 'route' in r and len(r['route']) == 2]
with open('routes.json', 'w') as file:
    json.dump(routes, file, indent=4)


In [5]:
# Download routes
# from google.colab import files
# files.download('routes.json')


In [6]:
# making actual route

import json
import random
import copy

# Set seed
random.seed(2)

# Read the JSON file
with open('cities.json', 'r') as file:
    data = json.load(file)

# Extract city names
cities = [item['city'] for item in data]

# Define merchandise types
merchandise_types = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

with open('routes.json', 'r') as file:
    routes = json.load(file)

act_routes = []

for route in routes:
    if len(route['route']) < 2:
        continue  # Skip routes with less than two cities

    for _ in range(10):
        act_route = copy.deepcopy(route)  # Use deepcopy to avoid modifying the original route

        # Randomly select a city to modify
        modified_city_index = random.randint(0, len(act_route['route']) - 1)
        modified_city = act_route['route'][modified_city_index]

        # Randomly decide whether to modify city
        if random.choice([True, False]):
            # Modify the selected city
            new_city = random.choice(cities)
            modified_city['from'] = new_city

            # If the modified city is the second city, modify it throughout the route
            if modified_city_index == 1:
                # Replace the modified city with a new city
                act_route['route'][modified_city_index]['from'] = new_city
                # Connect the cities before and after the replaced city
                act_route['route'][modified_city_index - 1]['to'] = new_city
                if modified_city_index + 1 < len(act_route['route']):
                    act_route['route'][modified_city_index]['to'] = act_route['route'][modified_city_index + 1]['from']

        # Randomly modify merchandise
        for city_route in act_route['route']:
            # Select random merchandise types
            selected_merchandise = random.sample(merchandise_types, random.randint(1, len(merchandise_types)))
            # Assign random quantities to selected merchandise
            city_route['merchandise'] = {item: random.randint(1, 5) for item in selected_merchandise}

        act_routes.append(act_route)

# Write the act routes list to a new JSON file
with open('act_routes.json', 'w') as file:
    json.dump(act_routes, file, indent=4)

In [7]:
# Download the actual routes
#from google.colab import files
#files.download('act_routes.json')

In [14]:
!pip install python-Levenshtein
import json
from Levenshtein import distance

def calculate_similarity(route1, route2):
    lev_distance = distance(route1, route2)
    max_length = max(len(route1), len(route2))
    similarity = 1 - lev_distance / max_length
    return similarity

def calculate_merchandise_similarity(merch1, merch2):
    lev_distance = distance(merch1, merch2)
    max_length = max(len(merch1), len(merch2))
    similarity = 1 - lev_distance / max_length
    return similarity

with open('routes.json', 'r') as file:
    routes = json.load(file)

with open('act_routes.json', 'r') as file:
    act_routes = json.load(file)

act_route = act_routes[11]['route']
act_route_str = ' '.join([f"{route['from']} {route['to']}" for route in act_route])

similarities = []

for standard_route in routes:
    standard_route_id = standard_route['id']
    standard_route = standard_route['route']
    standard_route_str = ' '.join([f"{route['from']} {route['to']}" for route in standard_route])

    route_similarity = calculate_similarity(standard_route_str, act_route_str)

    merchandise_similarity = 0
    for i, route in enumerate(act_route):
        act_merchandise = route['merchandise']
        standard_merchandise = standard_route[0]['merchandise']
        merchandise_similarity += calculate_merchandise_similarity(str(standard_merchandise), str(act_merchandise))
    merchandise_similarity /= len(standard_route)

    total_similarity = (route_similarity + merchandise_similarity) / 2
    similarities.append((standard_route_id, total_similarity))

similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
max_similarity = similarities[0]

print(f"Max similarity: Route {max_similarity[0]} with similarity {max_similarity[1]}")


Max similarity: Route 1 with similarity 0.6684228473998295


In [ ]:
#Edit distance calculation
# Create a new SparkContext
sc.stop()
sc = SparkContext("local", "RouteSimilarity")

# Load routes data
routes_rdd = sc.wholeTextFiles("routes.json").flatMap(lambda x: json.loads(x[1]))
act_routes_rdd = sc.wholeTextFiles("act_routes.json").flatMap(lambda x: json.loads(x[1]))

# Parse JSON and extract relevant data
routes = routes_rdd.map(lambda x: x)
act_routes = act_routes_rdd.map(lambda x: x)


# Extract standard route
standard_route = routes.filter(lambda x: x['id'] == 0).map(lambda x: x['route'][0])
standard_route_str = ' '.join([f"{route['from']} {route['to']}" for route in standard_route.collect()])


# Compute similarity for each act_route
similarities = act_routes.flatMap(lambda x: x['route']).map(lambda x: (x['id'], x['route'])).map(lambda x: (x[0], ' '.join([f"{route['from']} {route['to']}" for route in x[1]]))) \
    .map(lambda x: (x[0], calculate_similarity(standard_route_str, x[1]))) \
    .reduceByKey(lambda x, y: (x*0.8 + y*0.2) / 2)

# Compute similarity for route 1 only
similarity_1 = similarities.filter(lambda x: x[0] == 1)

# Collect the result and print it
for x in similarity_1.collect():
    print(f"Similarity for route {x[0]}: {x[1]}")

# Stop the SparkContext
sc.stop()

